In [1]:
import math 
import numpy as np
from numpy.typing import NDArray
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import random as rng
import networkx as nx
import matplotlib.pyplot as plt
import pickle
from typing import Union, List, Any
from qaoa_and_tdvp import QAOA, tdvp_optimize_qaoa, scipy_optimize, QAOAResult, Qobj
from MaxCut import MaxCut
from itertools import *

from qutip import tensor, basis, Qobj, expect, variance
from qutip.operators import qeye, sigmax, sigmay, sigmaz
from qutip.qip.operations import expand_operator, rz

import swifter

from benchmark import (
    get_all_connected,
    get_connected_rn_graph,
    get_rn_qubo,
    select_if_connected,
    Benchmark,
)
from qutip.parallel import parallel_map, serial_map

# convert data into DataFrame

In [58]:
with open('..//benchmarks//n10_data.p','rb') as file:
    df = pickle.load(file)

In [3]:
# with open("..//benchmarks//RK45_n10_p-1-5_Delta-1.p","rb") as file:
#     data: Benchmark = pickle.load(file)
#
# scipy_results = list()
# for d in data.results:
#     items = d.get("tdvp").__dict__.items()
#     for k,v in items:
#         d[k]=v
#         d["algorithm"] = "tdvp"
#     d["n"] = d["_qaoa"].n
#     dd = d.copy()
#     items = dd.get("scipy").__dict__.items()
#     for k,v in items:
#         dd[k]=v
#         dd["algorithm"] = "scipy"
#     dd["n"] = dd["_qaoa"].n
#     scipy_results.append(dd)
#
# data.results.extend(scipy_results)
#
# df = pd.DataFrame(data.results)
# df.pop("tdvp")
# df.pop("scipy")
# cols = df.columns.tolist()
# cols = cols[-1:] + cols[:-1]
# df = df[cols]
#
# df['steps'] = df.apply(lambda row: int(row['_num_steps']) if row.algorithm=="tdvp" else row['_num_fun_calls'], axis=1)
#
# df.insert(2,"algorithm",df.pop("algorithm"))
# df.insert(3,"_optimal_fun_value",df.pop("_optimal_fun_value"))
# df.insert(4,'steps', df.pop('steps'))
# df.insert(6,"_prob",df.pop("_prob"))
#
#
# df_tdvp = df[df.algorithm.eq("tdvp")]
# df_scipy = df[df.algorithm.eq("scipy")]
#
# df.rename(columns={"_optimal_fun_value":"C", "_prob":"prob", '_duration':'duration', '_optimal_state':'final_state', '_optimal_parameters':'final_parameters', '_num_gates':'num_gates','_qaoa':'qaoa'}, inplace=True)

In [59]:
df

,n,p,algorithm,expectation_of_approximation_ratio,approximation_ratio,best_cost_value,bin_answer,answer,gate_count,circuit_count,...,_parameter_path,_num_fun_calls,_num_steps,duration,final_state,final_parameters,_orig_result,num_gates,qaoa,groundspace
0,10,1,tdvp,0.154335,0.119048,-42.0,0010111010,186,27098552.0,81001.0,...,None,20,24.0,10.024673,"Quantum object: dims = [[2, 2, 2, 2, 2, 2, 2, ...","(0.8493584338845096, 0.9903816866539071)","{'t': [0.0, 0.11734882842785752, 0.67900509421...",15343,<qaoa_and_tdvp.QAOA object at 0x7fdb8bf8e320>,"[Quantum object: dims = [[2, 2, 2, 2, 2, 2, 2,..."
1,10,2,tdvp,0.131486,1.000000,-42.0,0000000000,0,34491304.0,303751.0,...,None,26,30.0,43.018689,"Quantum object: dims = [[2, 2, 2, 2, 2, 2, 2, ...","(0.897119778539972, 0.8061971496211522, 0.8727...","{'t': [0.0, 0.13675405274738983, 0.65103925319...",92061,<qaoa_and_tdvp.QAOA object at 0x7fdb8bf8e9e0>,"[Quantum object: dims = [[2, 2, 2, 2, 2, 2, 2,..."
2,10,3,tdvp,0.232425,1.000000,-42.0,0000000000,0,-79764144.0,749251.0,...,None,32,37.0,105.185971,"Quantum object: dims = [[2, 2, 2, 2, 2, 2, 2, ...","(0.8398889328861309, 1.0662341829920257, 0.880...","{'t': [0.0, 0.105645202418896, 0.3909706632630...",337710,<qaoa_and_tdvp.QAOA object at 0x7fdb8bf8f400>,"[Quantum object: dims = [[2, 2, 2, 2, 2, 2, 2,..."
3,10,4,tdvp,0.218356,0.595238,-42.0,0010011000,152,-484958592.0,1485001.0,...,None,38,44.0,244.791884,"Quantum object: dims = [[2, 2, 2, 2, 2, 2, 2, ...","(0.9248620211813144, 1.1375408981648616, 0.807...","{'t': [0.0, 0.09382788175041508, 0.40693789987...",893299,<qaoa_and_tdvp.QAOA object at 0x7fdb8bf8faf0>,"[Quantum object: dims = [[2, 2, 2, 2, 2, 2, 2,..."
4,10,5,tdvp,0.111067,0.523810,-42.0,0000110010,50,-858162940.0,1569376.0,...,None,26,31.0,328.186238,"Quantum object: dims = [[2, 2, 2, 2, 2, 2, 2, ...","(0.8464597461633876, 1.0563072405050975, 1.152...","{'t': [0.0, 0.10295826275518528, 0.44471718620...",1182725,<qaoa_and_tdvp.QAOA object at 0x7fdb8bf8c190>,"[Quantum object: dims = [[2, 2, 2, 2, 2, 2, 2,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,10,1,scipy,0.930109,1.000000,-30.0,0000000000,0,11880.0,99.0,...,None,98,NaN,6.088192,"Quantum object: dims = [[2, 2, 2, 2, 2, 2, 2, ...","[2.3626721807847217, -0.15676991175280042]","{'x': [2.3626721807847217, -0.1567699117528004...",6370,<qaoa_and_tdvp.QAOA object at 0x7fdb8b721de0>,"[Quantum object: dims = [[2, 2, 2, 2, 2, 2, 2,..."
996,10,2,scipy,0.153012,0.033333,-30.0,1011010100,724,112800.0,470.0,...,None,469,NaN,57.908112,"Quantum object: dims = [[2, 2, 2, 2, 2, 2, 2, ...","[0.8804501181503313, 1.822291141013481, 0.9091...","{'x': [0.8804501181503313, 1.822291141013481, ...",60970,<qaoa_and_tdvp.QAOA object at 0x7fdb8b722320>,"[Quantum object: dims = [[2, 2, 2, 2, 2, 2, 2,..."
997,10,3,scipy,0.211586,0.700000,-30.0,1001000100,580,159120.0,442.0,...,None,441,NaN,81.819692,"Quantum object: dims = [[2, 2, 2, 2, 2, 2, 2, ...","[0.7559838546298562, 0.39163966121792393, 1.29...","{'x': [0.7559838546298562, 0.39163966121792393...",85995,<qaoa_and_tdvp.QAOA object at 0x7fdb8b722830>,"[Quantum object: dims = [[2, 2, 2, 2, 2, 2, 2,..."
998,10,4,scipy,0.416602,0.800000,-30.0,0100000000,256,375840.0,783.0,...,None,782,NaN,194.630926,"Quantum object: dims = [[2, 2, 2, 2, 2, 2, 2, ...","[2.3414604674448056, 1.0393519895807521, 0.655...","{'x': [2.3414604674448056, 1.0393519895807521,...",203320,<qaoa_and_tdvp.QAOA object at 0x7fdb8b722d40>,"[Quantum object: dims = [[2, 2, 2, 2, 2, 2, 2,..."


# Compute performance measures

In [5]:
df.columns

Index(['n', 'p', 'algorithm', 'groundspace_overlap', 'groundspace_sharpness',
       'C', 'steps', 'delta_0', 'prob', 'tdvp_stepsize', 'tdvp_grad_tol',
       'tdvp_lineq_solver', '_message', '_optimizer_name', '_sucess',
       '_parameter_path', '_num_fun_calls', '_num_steps', 'duration',
       'final_state', 'final_parameters', '_orig_result', 'num_gates', 'qaoa',
       'groundspace'],
      dtype='object')

In [49]:
def prob(num:int, state:Qobj):
    n = state.shape[0]
    return abs(state.overlap(basis(n,num)))**2

def answer(state:Qobj):
    return max(range(state.shape[0]), key=lambda x:prob(x,state))

def bin_answer(state:Qobj):
    return format(answer(state),  f'0{len(state.dims[0])}b')

In [50]:
def best_cost_value(qaoa:QAOA):
    return qaoa.H.groundstate()[0]

def approximation_ratio(bin_answer:int, best_cost_value:float, qaoa:QAOA):
    i = list(int(b) for b in bin_answer)
    dims = [2 for _ in range(qaoa.n)]
    return expect(qaoa.H,basis(dims,i))/best_cost_value

def expectation_of_approximation_ratio(state:Qobj, best_cost_value:float, qaoa:QAOA):
    return (expect(qaoa.H, state))/best_cost_value


In [8]:
def num_metric_gates(n:int,p:int,qubo:NDArray):
    Q = np.count_nonzero(qubo)
    N = 2*p*(n+Q)
    C1 = 1/2 * n**4 + 2*n**3 + 7/2*n**2
    C2 = n**4 + 5/2*(n**3 + n**2)
    return N/p*C1*(1/6*p**3 - 1/6*p + p**2) + (2*C2-N*C1)*(1/2*p**2 + 1/2*p)

def num_gradient_gates(n:int,p:int,qubo:NDArray):
    Q = np.count_nonzero(qubo)
    N = 2*p*(n+Q)
    return (n**4 -2*n**3 +n**2)/4 * (2*N+4) + 3*n*(n*(n-1))/2*(2*(N+3)) + 2*n**2*(2*N+3)

def num_qaoa_gates(n:int,p:int, qubo:NDArray):
    Q = np.count_nonzero(qubo)
    return 2*p*(n+Q)

def circuit_count(steps:int, algorithm:str, p:int, n:int):
    if algorithm =='tdvp':
        return steps*(p**2 + p)*(1/8*n**4 +1/2*n**3 -5/8*n**2) + 1
    if algorithm =="scipy":
        return steps + 1

def gate_count(steps:int, algorithm:str, qaoa:QAOA, p:int, n:int):
    if algorithm=='tdvp':
        return steps*(num_metric_gates(n,p,qaoa.qubo) + num_gradient_gates(n,p,qaoa.qubo)) + num_qaoa_gates(n,p,qaoa.qubo)
    if algorithm=='scipy':
        return (1+steps)*num_qaoa_gates(n,p,qaoa.qubo)

In [9]:
def groundspace(qaoa:QAOA):
    eigenenergies, eigenstates = qaoa.H.eigenstates(sort='low')
    groundenergy = eigenenergies[0]

    num_of_eigenstates = 0
    for e in eigenenergies:
        if e==groundenergy:
            num_of_eigenstates +=1
        if e != groundenergy:
            break
    return eigenstates[:num_of_eigenstates]

def groundspace_overlap(groundstates:List[Qobj], state:Qobj):
    return  sum(abs(g.overlap(state))**2 for g in groundstates)

def groundspace_sharpness(groundstates:List[Qobj], state:Qobj):
    P_0 = sum(g*g.dag() for g in groundstates)
    return math.sqrt(variance(P_0,state))


In [10]:
# df['groundspace'] = df.swifter.apply(lambda row: groundspace(row.qaoa), axis=1)

In [11]:
# df.insert(3,'groundspace_overlap', df.swifter.apply(lambda row: groundspace_overlap(row['groundspace'], row['final_state']), axis=1))

Pandas Apply:   0%|          | 0/1000 [00:00<?, ?it/s]

ValueError: cannot insert groundspace_overlap, already exists

In [12]:
# df.insert(4, 'groundspace_sharpness', df.swifter.apply(lambda row: groundspace_sharpness(row['groundspace'], row['final_state']), axis=1))

Pandas Apply:   0%|          | 0/1000 [00:00<?, ?it/s]

ValueError: cannot insert groundspace_sharpness, already exists

In [14]:
df['steps']

0        24
1        30
2        37
3        44
4        31
       ... 
995      98
996     469
997     441
998     782
999    1000
Name: steps, Length: 1000, dtype: int64

In [15]:
df['circuit_count']= df.swifter.apply(lambda row: circuit_count(row['steps'], row['algorithm'], row['p'], row['n']), axis=1)

Pandas Apply:   0%|          | 0/1000 [00:00<?, ?it/s]

In [18]:
df['gate_count']= df.swifter.apply(lambda row: gate_count(row['steps'], row['algorithm'],row['qaoa'], row['p'], row['n']), axis=1)

Pandas Apply:   0%|          | 0/1000 [00:00<?, ?it/s]

In [19]:
df['answer'] = df.swifter.apply(lambda row: answer(row['final_state']), axis=1)

Pandas Apply:   0%|          | 0/1000 [00:00<?, ?it/s]

In [51]:
df['bin_answer'] = df.swifter.apply(lambda row: bin_answer(row['final_state']), axis=1)

Pandas Apply:   0%|          | 0/1000 [00:00<?, ?it/s]

In [52]:
df.insert(3, 'best_cost_value', df.swifter.apply(lambda row: best_cost_value(row['qaoa']), axis=1))

KeyboardInterrupt: 

In [53]:
df.insert(3, 'approximation_ratio', df.swifter.apply(lambda row: approximation_ratio(row['bin_answer'],row['best_cost_value'],row['qaoa']), axis=1))

Pandas Apply:   0%|          | 0/1000 [00:00<?, ?it/s]

In [56]:
df.insert(3, 'expectation_of_approximation_ratio', df.swifter.apply(lambda row: expectation_of_approximation_ratio(row['final_state'],row['best_cost_value'],row['qaoa']), axis=1))

Pandas Apply:   0%|          | 0/1000 [00:00<?, ?it/s]

In [57]:
with open('..//benchmarks//n10_data.p', 'wb') as file:
    pickle.dump(df,file)